<a href="https://colab.research.google.com/github/javierloren/worl-cup-2022-prediction/blob/main/project_futb_python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Extracción de datos de tablas. Grupos, selecciones

In [39]:
import pandas as pd
from string import ascii_uppercase as abecedario
import pickle
import bs4
from bs4 import BeautifulSoup
import requests
import lxml
from scipy.stats import poisson

Extraer tablas del grupo A hasta el H

In [40]:
tablas = pd.read_html('https://web.archive.org/web/20221115040351/https://en.wikipedia.org/wiki/2022_FIFA_World_Cup')
tablas[12]  # la primera tabla de los grupos está en la posición 12

,Pos,"Team.mw-parser-output .navbar{display:inline;font-size:88%;font-weight:normal}.mw-parser-output .navbar-collapse{float:left;text-align:left}.mw-parser-output .navbar-boxtext{word-spacing:0}.mw-parser-output .navbar ul{display:inline-block;white-space:nowrap;line-height:inherit}.mw-parser-output .navbar-brackets::before{margin-right:-0.125em;content:""[ ""}.mw-parser-output .navbar-brackets::after{margin-left:-0.125em;content:"" ]""}.mw-parser-output .navbar li{word-spacing:-0.125em}.mw-parser-output .navbar a>span,.mw-parser-output .navbar a>abbr{text-decoration:inherit}.mw-parser-output .navbar-mini abbr{font-variant:small-caps;border-bottom:none;text-decoration:none;cursor:inherit}.mw-parser-output .navbar-ct-full{font-size:114%;margin:0 7em}.mw-parser-output .navbar-ct-mini{font-size:114%;margin:0 4em}vte",Pld,W,D,L,GF,GA,GD,Pts,Qualification
0,1,Qatar (H),0,0,0,0,0,0,0,0,Advance to knockout stage
1,2,Ecuador,0,0,0,0,0,0,0,0,Advance to knockout stage
2,3,Senegal,0,0,0,0,0,0,0,0,NaN
3,4,Netherlands,0,0,0,0,0,0,0,0,NaN


In [41]:
tablas[19].columns # Modificar columnas. La columna 'qualification' eliminar, columna 'teamvte' dejarla solo como team

Index(['Pos', 'Teamvte', 'Pld', 'W', 'D', 'L', 'GF', 'GA', 'GD', 'Pts',
       'Qualification'],
      dtype='object')

In [42]:
tablas = pd.read_html('https://web.archive.org/web/20221115040351/https://en.wikipedia.org/wiki/2022_FIFA_World_Cup')

dict_grupos = {}
for letra, i in zip(abecedario, range(12,68,7)):
  df = tablas[i]
  df.rename(columns={df.columns[1] : 'team'}, inplace = True)
  df.pop('Qualification')
  dict_grupos[f'Grupo {letra}'] = df


In [43]:
dict_grupos['Grupo H']

,Pos,team,Pld,W,D,L,GF,GA,GD,Pts
0,1,Portugal,0,0,0,0,0,0,0,0
1,2,Ghana,0,0,0,0,0,0,0,0
2,3,Uruguay,0,0,0,0,0,0,0,0
3,4,South Korea,0,0,0,0,0,0,0,0


In [44]:
with open ('dict_table', 'wb') as output:
  pickle.dump(dict_grupos, output)

Histórico datos de mundiales

In [45]:
years = [1930,1934,1938,1950,1958,1962,1966,1970,1974,1978,1982,1986,1990,1998,2002,2010,2014,2018]

In [48]:
def get_matches(year):
    web = f'https://en.wikipedia.org/wiki/{year}_FIFA_World_Cup'

    response = requests.get(web)
    content = response.text
    soup = BeautifulSoup(content, 'lxml')

    matches = soup.find_all('div', class_='footballbox')


    home = []
    score = []
    away = []

    for match in matches:
      home.append(match.find('th', class_= 'fhome').get_text())
      score.append(match.find('th', class_= 'fscore').get_text())
      away.append(match.find('th', class_= 'faway').get_text())


    dict_football = {'home': home, 'score': score, 'away': away}
    df_football = pd.DataFrame(dict_football)
    df_football['year'] = year
    return df_football



mundiales = [get_matches(year) for year in years]
df_mundiales = pd.concat(mundiales, ignore_index=True)
df_mundiales.to_csv('fifa_worldcup_historical_data.csv', index=False)


df_fixture = get_matches('2022')

df_fixture = df_mundiales.to_csv('fifa_worldcup_fixture_data.csv', index=False)



Limpieza y transformación de datos

In [50]:
df_data_historica = pd.read_csv('fifa_worldcup_historical_data.csv')
df_fixture = pd.read_csv('fifa_worldcup_fixture.csv')
df_missing = pd.read_csv('fifa_worldcup_missing_data.csv')

In [51]:
df_fixture['home']= df_fixture['home'].str.strip()
df_fixture['away']= df_fixture['away'].str.strip()

In [52]:
df_missing[df_missing['home'].isnull()]

,home,score,away,year
396,NaN,NaN,NaN,2010
397,NaN,NaN,NaN,2010
398,NaN,NaN,NaN,2010
399,NaN,NaN,NaN,2010
400,NaN,NaN,NaN,2010
...,...,...,...,...
455,NaN,NaN,NaN,2010
456,NaN,NaN,NaN,2010
457,NaN,NaN,NaN,2010
458,NaN,NaN,NaN,2010


In [53]:
df_missing.dropna(inplace=True)
df_missing[df_missing['home'].isnull()]

,home,score,away,year


In [54]:
df_data_historica= pd.concat([df_missing, df_data_historica], ignore_index=True)
df_data_historica.drop_duplicates(inplace=True)
df_data_historica.sort_values('year', inplace=True)
df_data_historica

,home,score,away,year
444,France,4–1,Mexico,1930
461,Uruguay,4–2,Argentina,1930
460,Uruguay,6–1,Yugoslavia,1930
459,Argentina,6–1,United States,1930
458,Paraguay,1–0,Belgium,1930
...,...,...,...,...
1165,Brazil,2–0,Costa Rica,2018
1166,Serbia,1–2,Switzerland,2018
1167,Serbia,0–2,Brazil,2018
1169,Germany,0–1,Mexico,2018


Se busca Sweden y Austria porque tienen una circunstancia particular que es la de un resultado por incomparecencia


In [55]:
index_eliminar= df_data_historica[df_data_historica['home'].str.contains('Sweden') &
                  df_data_historica['away'].str.contains('Austria')].index

In [56]:
df_data_historica.drop(index=index_eliminar, inplace=True)

In [57]:
df_data_historica[df_data_historica['home'].str.contains('Sweden') &
                  df_data_historica['away'].str.contains('Austria')].index

Int64Index([], dtype='int64')

En columna 'score' además del propio resultado aparecen en algunas filas texto

In [58]:
# df_data_historica[df_data_historica['score'].str.contains('[^\d–]')]
df_data_historica['score']= df_data_historica['score'].str.replace('[^\d–]', '', regex=True)

In [59]:
df_data_historica[df_data_historica['score'].str.contains('[^\d–]')]

,home,score,away,year


In [60]:
df_data_historica['home']= df_data_historica['home'].str.strip()
df_data_historica['away']= df_data_historica['away'].str.strip()

In [21]:
#print(df_data_historica['score'].head())



In [61]:
df_data_historica[['homegoals', 'awaygoals']]= df_data_historica['score'].str.split('–', expand=True)


In [62]:
df_data_historica.drop('score', axis=1, inplace=True)

In [63]:
df_data_historica

,home,away,year,homegoals,awaygoals
444,France,Mexico,1930,4,1
461,Uruguay,Argentina,1930,4,2
460,Uruguay,Yugoslavia,1930,6,1
459,Argentina,United States,1930,6,1
458,Paraguay,Belgium,1930,1,0
...,...,...,...,...,...
1165,Brazil,Costa Rica,2018,2,0
1166,Serbia,Switzerland,2018,1,2
1167,Serbia,Brazil,2018,0,2
1169,Germany,Mexico,2018,0,1


In [64]:
df_data_historica= df_data_historica.astype({'homegoals': int, 'awaygoals':int })

In [65]:
df_data_historica.dtypes

home         object
away         object
year          int64
homegoals     int64
awaygoals     int64
dtype: object

In [66]:
df_data_historica['Totalgoals']= df_data_historica['homegoals'] + df_data_historica['awaygoals']
df_data_historica

,home,away,year,homegoals,awaygoals,Totalgoals
444,France,Mexico,1930,4,1,5
461,Uruguay,Argentina,1930,4,2,6
460,Uruguay,Yugoslavia,1930,6,1,7
459,Argentina,United States,1930,6,1,7
458,Paraguay,Belgium,1930,1,0,1
...,...,...,...,...,...,...
1165,Brazil,Costa Rica,2018,2,0,2
1166,Serbia,Switzerland,2018,1,2,3
1167,Serbia,Brazil,2018,0,2,2
1169,Germany,Mexico,2018,0,1,1


In [67]:
df_data_historica.to_csv('clean_fifa_worldcup_historical_data.csv', index=False)
df_fixture.to_csv('clean_fifa_worldcup_fixture.csv', index=False)

Creación del modelo. Evaluación

In [68]:
dict_table = pickle.load(open('dict_table', 'rb'))
df_historical_data = pd.read_csv('clean_fifa_worldcup_matches.csv')
df_fixture = pd.read_csv('clean_fifa_worldcup_fixture.csv')



Team Strength para describir la capacidad goleadora y la defensiva

In [69]:
df_historical_data

,HomeTeam,AwayTeam,Year,HomeGoals,AwayGoals,TotalGoals
0,France,Mexico,1930,4,1,5
1,Uruguay,Argentina,1930,4,2,6
2,Uruguay,Yugoslavia,1930,6,1,7
3,Argentina,United States,1930,6,1,7
4,Paraguay,Belgium,1930,1,0,1
...,...,...,...,...,...,...
895,Brazil,Costa Rica,2018,2,0,2
896,Serbia,Switzerland,2018,1,2,3
897,Serbia,Brazil,2018,0,2,2
898,France,Peru,2018,1,0,1


In [70]:
df_home = df_historical_data[['HomeTeam', 'HomeGoals', 'AwayGoals']]
df_away = df_historical_data[['AwayTeam', 'HomeGoals', 'AwayGoals']]

df_home = df_home.rename(columns={'HomeTeam':'Team', 'HomeGoals': 'GoalsScored', 'AwayGoals': 'GoalsConceded'})
df_away = df_away.rename(columns={'AwayTeam':'Team', 'HomeGoals': 'GoalsConceded', 'AwayGoals': 'GoalsScored'})

df_team_strength = pd.concat([df_home, df_away], ignore_index=True).groupby(['Team']).mean()
df_team_strength

,GoalsScored,GoalsConceded
Team,,
Algeria,1.000000,1.461538
Angola,0.333333,0.666667
Argentina,1.691358,1.148148
Australia,0.812500,1.937500
Austria,1.482759,1.620690
...,...,...
Uruguay,1.553571,1.321429
Wales,0.800000,0.800000
West Germany,2.112903,1.241935


In [71]:
def predict_points(home, away):
    if home in df_team_strength.index and away in df_team_strength.index:
        # goals_scored * goals_conceded
        lamb_home = df_team_strength.at[home,'GoalsScored'] * df_team_strength.at[away,'GoalsConceded']
        lamb_away = df_team_strength.at[away,'GoalsScored'] * df_team_strength.at[home,'GoalsConceded']
        prob_home, prob_away, prob_draw = 0, 0, 0
        for x in range(0,11): #number of goals home team
            for y in range(0, 11): #number of goals away team
                p = poisson.pmf(x, lamb_home) * poisson.pmf(y, lamb_away)
                if x == y:
                    prob_draw += p
                elif x > y:
                    prob_home += p
                else:
                    prob_away += p

        points_home = 3 * prob_home + prob_draw
        points_away = 3 * prob_away + prob_draw
        return (points_home, points_away)
    else:
        return (0, 0)

In [72]:
print(predict_points('England', 'United States'))
print(predict_points('Argentina', 'Mexico'))
print(predict_points('Qatar (H)', 'Ecuador')) # Qatar vs Team X -> 0 points to both

(2.2356147635326007, 0.5922397535606193)
(2.3129151525530505, 0.5378377125059863)
(0, 0)


probar funcion


In [73]:
predict_points('Argentina', 'Mexico')
predict_points('Argentina', 'Spain')
print(predict_points('Qatar (H)', 'Ecuador')) # Qatar vs Team X -> 0 points to both

(0, 0)


Predicción


In [74]:
df_fixture_group_48 = df_fixture[:48].copy()
df_fixture_knockout = df_fixture[48:56].copy()
df_fixture_quarter = df_fixture[56:60].copy()
df_fixture_semi = df_fixture[60:62].copy()
df_fixture_final = df_fixture[62:].copy()

In [75]:
for group in dict_table:
    teams_in_group = dict_table[group]['Team'].values
    df_fixture_group_6 = df_fixture_group_48[df_fixture_group_48['home'].isin(teams_in_group)]
    for index, row in df_fixture_group_6.iterrows():
        home, away = row['home'], row['away']
        points_home, points_away = predict_points(home, away)
        dict_table[group].loc[dict_table[group]['Team'] == home, 'Pts'] += points_home
        dict_table[group].loc[dict_table[group]['Team'] == away, 'Pts'] += points_away

    dict_table[group] = dict_table[group].sort_values('Pts', ascending=False).reset_index()
    dict_table[group] = dict_table[group][['Team', 'Pts']]
    dict_table[group] = dict_table[group].round(0)

In [76]:
dict_table['Group A']

,Team,Pts
0,Netherlands,4.0
1,Senegal,2.0
2,Ecuador,2.0
3,Qatar (H),0.0


In [77]:
df_fixture_knockout

,home,score,away,year
48,Winners Group A,Match 49,Runners-up Group B,2022
49,Winners Group C,Match 50,Runners-up Group D,2022
50,Winners Group D,Match 52,Runners-up Group C,2022
51,Winners Group B,Match 51,Runners-up Group A,2022
52,Winners Group E,Match 53,Runners-up Group F,2022
53,Winners Group G,Match 54,Runners-up Group H,2022
54,Winners Group F,Match 55,Runners-up Group E,2022
55,Winners Group H,Match 56,Runners-up Group G,2022


In [78]:
for group in dict_table:
    group_winner = dict_table[group].loc[0, 'Team']
    runners_up = dict_table[group].loc[1, 'Team']
    df_fixture_knockout.replace({f'Winners {group}':group_winner,
                                 f'Runners-up {group}':runners_up}, inplace=True)

df_fixture_knockout['winner'] = '?'
df_fixture_knockout

,home,score,away,year,winner
48,Netherlands,Match 49,Wales,2022,?
49,Argentina,Match 50,Denmark,2022,?
50,France,Match 52,Poland,2022,?
51,England,Match 51,Senegal,2022,?
52,Germany,Match 53,Belgium,2022,?
53,Brazil,Match 54,Uruguay,2022,?
54,Croatia,Match 55,Spain,2022,?
55,Portugal,Match 56,Switzerland,2022,?


In [79]:
def get_winner(df_fixture_updated):
    for index, row in df_fixture_updated.iterrows():
        home, away = row['home'], row['away']
        points_home, points_away = predict_points(home, away)
        if points_home > points_away:
            winner = home
        else:
            winner = away
        df_fixture_updated.loc[index, 'winner'] = winner
    return df_fixture_updated

In [80]:
get_winner(df_fixture_knockout)

,home,score,away,year,winner
48,Netherlands,Match 49,Wales,2022,Netherlands
49,Argentina,Match 50,Denmark,2022,Argentina
50,France,Match 52,Poland,2022,France
51,England,Match 51,Senegal,2022,England
52,Germany,Match 53,Belgium,2022,Germany
53,Brazil,Match 54,Uruguay,2022,Brazil
54,Croatia,Match 55,Spain,2022,Spain
55,Portugal,Match 56,Switzerland,2022,Portugal


In [81]:
def update_table(df_fixture_round_1, df_fixture_round_2):
    for index, row in df_fixture_round_1.iterrows():
        winner = df_fixture_round_1.loc[index, 'winner']
        match = df_fixture_round_1.loc[index, 'score']
        df_fixture_round_2.replace({f'Winners {match}':winner}, inplace=True)
    df_fixture_round_2['winner'] = '?'
    return df_fixture_round_2

In [82]:
update_table(df_fixture_knockout, df_fixture_quarter)

,home,score,away,year,winner
56,Germany,Match 58,Brazil,2022,?
57,Netherlands,Match 57,Argentina,2022,?
58,Spain,Match 60,Portugal,2022,?
59,England,Match 59,France,2022,?


In [83]:
get_winner(df_fixture_quarter)

,home,score,away,year,winner
56,Germany,Match 58,Brazil,2022,Brazil
57,Netherlands,Match 57,Argentina,2022,Netherlands
58,Spain,Match 60,Portugal,2022,Portugal
59,England,Match 59,France,2022,France


In [84]:
update_table(df_fixture_quarter, df_fixture_semi)

,home,score,away,year,winner
60,Netherlands,Match 61,Brazil,2022,?
61,France,Match 62,Portugal,2022,?


In [85]:
get_winner(df_fixture_semi)

,home,score,away,year,winner
60,Netherlands,Match 61,Brazil,2022,Brazil
61,France,Match 62,Portugal,2022,France


In [86]:
update_table(df_fixture_semi, df_fixture_final)

,home,score,away,year,winner
62,Losers Match 61,Match 63,Losers Match 62,2022,?
63,Brazil,Match 64,France,2022,?


In [87]:
get_winner(df_fixture_final)

,home,score,away,year,winner
62,Losers Match 61,Match 63,Losers Match 62,2022,Losers Match 62
63,Brazil,Match 64,France,2022,Brazil
